<a href="https://colab.research.google.com/github/Vyoma-garg/Natural-Language-Processing/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Getting dataset**

In [ ]:
import pandas as pd
import gzip
false=False
true=True

def parse(path):
  g = gzip.open('/content/Magazine_Subscriptions.json.gz', 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('Magazine_Subscriptions.json.gz')

Total Number of Magazine Subscriptions reviews 

In [ ]:
len(df)

89689

Total Number of reviews taken in consideration 

In [ ]:
df=df[:25000]
len(df)

25000

In [ ]:
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment,Binary_Sentiment
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN,Positive,1.0
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN,Positive,1.0
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN,Negative,0.0
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN,Positive,1.0
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN,Positive,1.0


Adding a column for Positive and Negative Sentiment

In [ ]:
df.loc[df['overall']<=3, 'Sentiment']='Negative'
df.loc[df['overall']>3, 'Sentiment']='Positive'
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN,Positive
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN,Positive
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN,Negative
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN,Positive
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN,Positive


Adding a column for Sentiments in binary form

In [ ]:
df.loc[df['Sentiment']=='Negative', 'Binary_Sentiment']=0
df.loc[df['Sentiment']=='Positive', 'Binary_Sentiment']=1
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image,Sentiment,Binary_Sentiment
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN,Positive,1.0
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN,Positive,1.0
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN,Negative,0.0
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN,Positive,1.0
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN,Positive,1.0


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopset=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
corpus = df["reviewText"].fillna(' ').values.tolist()
rating=df['overall'].values.tolist()
sentiment=df['Sentiment'].values.tolist()
bin_sentiment=df['Binary_Sentiment'].values.tolist()
print(bin_sentiment)

[1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

**1. (a) CountVectorizer**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer 

# Create a Vectorizer Object 
vectorizer = CountVectorizer(lowercase=True, stop_words=stopset,) 

#tokenize and build vocab 
X_cv=vectorizer.fit_transform(corpus)  #25000x25711
print(X_cv[0])
print(vectorizer.get_feature_names())

# Printing the identified Unique words along with their indices 
print("Vocabulary: ", vectorizer.vocabulary_)

data = vectorizer.vocabulary_

#pd.Series().to_frame(vectorizer.vocabulary)
dic= pd.DataFrame.from_dict([data])
dic

  (0, 4956)	1
  (0, 7929)	2
  (0, 14304)	3
  (0, 25057)	1
  (0, 20819)	1
  (0, 13967)	1
  (0, 19067)	1
  (0, 25574)	1
  (0, 20041)	1
  (0, 8192)	2
  (0, 16430)	1
  (0, 3166)	2
  (0, 3696)	1
  (0, 2649)	1
  (0, 21913)	1
  (0, 15765)	1
  (0, 16700)	1
  (0, 1530)	1
  (0, 905)	1
  (0, 24387)	1
  (0, 2648)	1
  (0, 13682)	1
  (0, 12985)	1
  (0, 17908)	1
  (0, 12430)	1
  :	:
  (0, 22226)	1
  (0, 13955)	1
  (0, 24827)	1
  (0, 14502)	1
  (0, 7340)	1
  (0, 14407)	1
  (0, 14314)	1
  (0, 7799)	1
  (0, 15898)	1
  (0, 11895)	1
  (0, 6960)	1
  (0, 4627)	1
  (0, 7779)	1
  (0, 15039)	1
  (0, 2752)	1
  (0, 2399)	1
  (0, 3479)	1
  (0, 2035)	1
  (0, 24629)	1
  (0, 8018)	1
  (0, 23781)	1
  (0, 12433)	1
  (0, 14308)	1
  (0, 9883)	1
  (0, 11162)	1
['00', '000', '0000000001', '00005', '0006058434', '00s', '01', '0195366824', '02', '02081', '0235', '02809', '03', '0320', '0399146113', '04', '0465061796', '05', '05465', '0595472273', '06', '07', '0714838284', '0736063684', '08', '0801', '0812972821', '081297666

,computer,enthusiast,maxpc,welcome,sight,mailbox,remember,years,savorying,every,page,boot,called,beginning,still,obcessed,pc,anyone,advanced,users,beginners,looking,knowledge,profit,issue,icing,cake,subscription,comes,cd,rom,packed,demos,utilities,useful,apps,helpful,blessed,broadband,connections,...,bombarding,logo,referential,nikey,funner,lon,b00005n7sm,liv,christensen,misshapes,cobrasnake,dancing,doodles,visitbritain,scottish,duke,middleton,moniker,witchcraft,boscastle,witches,warlocks,paraphernalia,oak,regency,cheltenham,centre,hedgerows,irregularly,alluded,transitions,manson,rory,chivers,monet,walled,wye,resurrection,cultivation,analyzes
0,4956,7929,14304,25057,20819,13967,19067,25574,20041,8192,16430,3166,3696,2649,21913,15765,16700,1530,905,24387,2648,13682,12985,17908,12430,11430,3673,22224,4759,4008,19694,16418,6285,24408,24380,1650,10870,2955,3412,5088,...,3103,13653,18825,15506,9610,13659,2200,13576,4333,14739,4614,5925,7056,24700,20179,7287,14582,14889,25290,3194,25291,24891,16551,15755,18889,4223,4058,10821,12385,1226,23581,14093,19723,4296,14874,24843,25516,19393,5776,1390


**1.(b) TF-IDF**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

#create transform
vectorizer2=TfidfVectorizer(use_idf=True, lowercase=True, stop_words=stopset)

#tokenize and build vocab
X_tfidf=vectorizer2.fit_transform(corpus) 
print(X_tfidf.shape)

#printing idf values
print(vectorizer2.idf_)

df5 = pd.DataFrame(X_tfidf[0].T.todense(), index=vectorizer2.get_feature_names(), columns=["TF-IDF"])
df5 = df5.sort_values('TF-IDF', ascending=False)
print (df5.head(25))

(25000, 25711)
[ 6.0768151   6.85000498 10.43352392 ... 10.43352392 10.43352392
 10.43352392]
               TF-IDF
maxpc        0.373384
boot         0.236606
computing    0.222767
enthusiast   0.208770
cd           0.190191
obcessed     0.145426
divx         0.145426
savorying    0.145426
codecs       0.145426
encoding     0.139774
merge        0.135764
icing        0.132654
broadband    0.130113
rom          0.126103
blessed      0.124461
utilities    0.122993
formerly     0.121664
connections  0.120452
bang         0.119336
audio        0.118303
cake         0.116442
sight        0.112651
profit       0.109149
welcome      0.109149
buck         0.107223


**1(c)Hashing Vectorizer**

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer
sentences = corpus
vectorizer = HashingVectorizer(norm = None, n_features = 25711)
sentence_vectors = vectorizer.fit_transform(sentences)
print(sentence_vectors[0])
#sent_arr =  sentence_vectors.toarray()

  (0, 18)	-1.0
  (0, 290)	-1.0
  (0, 471)	1.0
  (0, 576)	-1.0
  (0, 603)	-1.0
  (0, 1541)	1.0
  (0, 1607)	1.0
  (0, 2045)	1.0
  (0, 2287)	3.0
  (0, 2839)	-1.0
  (0, 2908)	1.0
  (0, 3259)	2.0
  (0, 3467)	-1.0
  (0, 3764)	3.0
  (0, 3961)	-2.0
  (0, 4273)	1.0
  (0, 4594)	1.0
  (0, 4761)	-1.0
  (0, 4834)	-1.0
  (0, 4872)	-1.0
  (0, 5219)	1.0
  (0, 5306)	2.0
  (0, 5315)	1.0
  (0, 5490)	1.0
  (0, 5602)	2.0
  :	:
  (0, 21125)	-1.0
  (0, 21275)	-1.0
  (0, 21354)	-1.0
  (0, 21706)	1.0
  (0, 22075)	-1.0
  (0, 23130)	-1.0
  (0, 23141)	1.0
  (0, 23190)	1.0
  (0, 23421)	-1.0
  (0, 23602)	-1.0
  (0, 23688)	-1.0
  (0, 23736)	-1.0
  (0, 24008)	-1.0
  (0, 24315)	-1.0
  (0, 24353)	-1.0
  (0, 24689)	1.0
  (0, 24877)	1.0
  (0, 25004)	1.0
  (0, 25142)	-5.0
  (0, 25225)	-2.0
  (0, 25309)	-1.0
  (0, 25516)	-1.0
  (0, 25568)	3.0
  (0, 25591)	1.0
  (0, 25619)	-2.0


## **2. Sentiment Analysis Using following models**

**2.(a)Naive Bayes**

In [ ]:
import pandas as pd
from sklearn import naive_bayes
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, confusion_matrix

using CountVectorizer

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_cv,bin_sentiment,random_state=40)
print(X_test.shape)
print(X_test[:,1].shape)
clf1=naive_bayes.MultinomialNB()
clf1.fit(X_train,Y_train)
yp=clf1.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

(6250, 25711)
(6250, 1)
[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 

Using TF-IDF

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_tfidf,bin_sentiment,random_state=40)
print(X_test.shape)
print(X_test[:,1].shape)
clf1=naive_bayes.MultinomialNB()
clf1.fit(X_train,Y_train)
yp=clf1.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

(6250, 25711)
(6250, 1)
[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 

**2(b)Decision Tree**

In [ ]:
from sklearn import tree

Using CountVectorizer

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_cv,bin_sentiment,random_state=40)

clf3=tree.DecisionTreeClassifier()
clf3.fit(X_train,Y_train)
yp=clf3.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0,

Using TF-IDF

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_tfidf,bin_sentiment,random_state=40) #25% test data and 75% train data

clf3=tree.DecisionTreeClassifier()
clf3.fit(X_train,Y_train)
yp=clf3.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0,

**2(c)Logistic Regression**

In [ ]:
from sklearn.linear_model import LogisticRegression

Using CountVectorizer

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_cv,bin_sentiment,random_state=40)
print(X_test.shape)
print(X_test[:,1].shape)
clf2=LogisticRegression()
clf2.fit(X_train,Y_train)
yp=clf2.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

(6250, 25711)
(6250, 1)
[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Using TF-IDF

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X_tfidf,bin_sentiment,random_state=40)
print(X_test.shape)
print(X_test[:,1].shape)
clf2=LogisticRegression()
clf2.fit(X_train,Y_train)
yp=clf2.predict(X_test)
print(Y_test)
print(yp)
print(accuracy_score(Y_test,yp))
print(f1_score(Y_test,yp))
print(confusion_matrix(Y_test,yp))

(6250, 25711)
(6250, 1)
[1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 